In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [12]:
def unify(expr1,expr2):
    if expr1 == expr2:
        return []

    if isConstant(expr1) and isConstant(expr2):
        if expr1 != expr2:
            return False

    if isConstant(expr1):
        return [(expr1, expr2)]

    if isConstant(expr2):
        return [(expr2, expr1)]

    if isVariable(expr1):
        if checkOccurs(expr1, expr2):
            return False
        else:
            return [(expr2, expr1)]

    if isVariable(expr2):
        if checkOccurs(expr2, expr1):
            return False
        else:
            return [(expr1, expr2)]

    if getInitialPredicate(expr1) != getInitialPredicate(expr2):
        print("Cannot be unified")
        return False

    atbCount1 = len(getAttributes(expr1))
    atbCount2 = len(getAttributes(expr2))
    if atbCount1 != atbCount2:
        return False

    head1 = getFirstPart(expr1)
    head2 = getFirstPart(expr2)
    initialSub = unify(head1, head2)
    if not initialSub:
        return False
    if atbCount1 == 1:
        return initialSub

    tail1 = getRemainingPart(expr1)
    tail2 = getRemainingPart(expr2)

    if initialSub != []:
        tail1 = apply(tail1, initialSub)
        tail2 = apply(tail2, initialSub)

    remainingSub = unify(tail1, tail2)
    if not remainingSub:
        return False

    initialSub.extend(remainingSub)
    res = []
    for tup in initialSub:
      st = ' / '.join(tup)
      res.append(st)

    return res

In [13]:
def checkOccurs(var, expr):
    if expr.find(var) == -1:
        return False
    return True

def getFirstPart(expr):
    attr = getAttributes(expr)
    return attr[0]

def getRemainingPart(expr):
    predicate = getInitialPredicate(expr)
    attr = getAttributes(expr)
    newExpr = predicate + "(" + ",".join(attr[1:]) + ")"
    return newExpr

def getAttributes(expr):
    expr = expr.split("(")[1:]
    expr = "(".join(expr)
    expr = expr[:-1]
    expr = re.split("(?<!\(.),(?!.\))", expr)
    return expr

def getInitialPredicate(expr):
    return expr.split("(")[0]

def isConstant(char):
    return char.isupper() and len(char) == 1

def isVariable(char):
    return char.islower() and len(char) == 1

def replaceAttributes(expr, old, new):
    attr = getAttributes(expr)
    for index, val in enumerate(attr):
        if val == old:
            attr[index] = new
    predicate = getInitialPredicate(expr)
    return predicate + "(" + ",".join(attr) + ")"

def apply(expr, subs):
    for sub in subs:
        new, old = sub  #substitution is a tuple of 2 values (new, old)
        expr = replaceAttributes(expr, old, new)
    return expr


In [14]:
expr1 = "Likes(Ram,y)"
expr2 = "Likes(x,Raj)"
subs = unify(expr1, expr2)
print("Substitutions:")
print(subs)

Substitutions:
['Ram / x', 'Raj / y']
